In [5]:
import sys
sys.path.append("/Users/mikelgallo/repos/CDS_final_pokemon") # main path to repo

In [4]:
import pandas as pd
from sklearn.metrics import roc_auc_score
#Own library
import library_final
#from library_final.pokedex import load_class as ld
#from library_final.pokedex import prepro_class as pr
#from library_final.pokedex import feature_class as ft
#from library_final.pokedex import model_class as mc


############DATABASE SQL
import pandas.io.sql as sqlio
import psycopg2
import json
from sqlalchemy import create_engine
from pydataset import data

In [7]:
df = pd.read_csv("/Users/mikelgallo/repos/CDS_final_pokemon/raw_data/ProcessedData.csv")

In [8]:
df

,Unnamed: 0,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,...,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water,Unnamed: 37,Type
0,0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,...,0,0,0,1,0,0,0,0,0,Grass
1,1,2,Ivysaur,Grass,Poison,60,62,63,80,80,...,0,0,0,1,0,0,0,0,0,Grass
2,2,3,Venusaur,Grass,Poison,80,82,83,100,100,...,0,0,0,1,0,0,0,0,0,Grass
3,3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,...,0,0,0,1,0,0,0,0,0,Grass
4,4,5,Charmander,Fire,NaN,39,52,43,60,50,...,0,0,0,0,0,0,0,0,1,Fire
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,795,796,Diancie,Rock,Fairy,50,100,150,100,150,...,0,0,0,0,0,1,0,0,0,Fairy
796,796,797,Mega Diancie,Rock,Fairy,50,160,110,160,110,...,0,0,0,0,0,1,0,0,0,Fairy
797,797,798,Hoopa Confined,Psychic,Ghost,80,110,60,150,130,...,0,0,0,0,1,0,0,0,0,Ghost
798,798,799,Hoopa Unbound,Psychic,Dark,80,160,60,170,130,...,0,0,0,0,1,0,0,0,0,Dark


#### Load Train and Test data

In [2]:
loader = ld.diabetes_loader_jupyter()

In [3]:
train, test = loader.train_and_test_data()

#### Preprocessing

In [4]:
# Create missing remover objects for train and test
missing_remover = pr.DropMissingValuesGenderEthnicity(train)
missing_remover_t = pr.DropMissingValuesGenderEthnicity(test)

In [5]:
# Remove missing rows from columns Age, gender, Ethnicity
train_clean = missing_remover.process()
test_clean = missing_remover_t.process()

In [6]:
# Create Imputation of missing values object for train and test
missing_imputator = pr.FillMissingValuesWithMeanHeightWeight(train_clean)
missing_imputator_t = pr.FillMissingValuesWithMeanHeightWeight(test_clean)

In [7]:
# Impute missing values for height and weight
train_imp = missing_imputator.process()
test_imp = missing_imputator_t.process()

#### Feature transformation

In [8]:
# One hot enco object
feature_trans = ft.One_hot_enc(train_imp)
feature_trans_t = ft.One_hot_enc(test_imp)

In [9]:
# Create new dummy columns based on icu_admit_source
train_new_f = feature_trans.one_hot_enc('icu_admit_source')
test_new_f = feature_trans_t.one_hot_enc('icu_admit_source')

object
object


In [10]:
# Create object for replacing boolean to 1-0
boolean_trans = ft.replace_booleans(train_new_f)
boolean_trans_t = ft.replace_booleans(test_new_f)

In [11]:
# Transform gender variable to binary
final_train = boolean_trans.input_mean_weight('gender')
final_test = boolean_trans_t.input_mean_weight('gender')

#### Model train and prediction

In [12]:
# Feature columns
X = ["age", "height", "weight", "aids", "cirrhosis", "hepatic_failure", 
     "immunosuppression", "leukemia", "lymphoma", "solid_tumor_with_metastasis","gender",
     'icu_admit_source_Accident & Emergency', 'icu_admit_source_Floor', 'icu_admit_source_Operating Room / Recovery', 'icu_admit_source_Other Hospital', 'icu_admit_source_Other ICU']
# Target column
Y = "diabetes_mellitus"

In [13]:
# Create logistic model class
logistic_model = mc.CustomModel(final_train,X,Y,'Logistic')

In [14]:
#Train model
logistic_model.train()

In [16]:
# Predict test values
probabilities, predictions = logistic_model.prediction(final_test[X])

In [37]:
# Add a new column "predictions" to final_test dataset
final_test['probabilities'] = probabilities.tolist() # Assuming you're interested in the probabilities of the positive class
final_test['predictions'] = predictions.tolist()  # Final prediction [0,1]

In [42]:
# Compute the ROC AUC score (substitute )
roc_auc = roc_auc_score(final_test['diabetes_mellitus'], final_test['predictions'])
print(f"ROC AUC Score: {roc_auc}")

ROC AUC Score: 0.5186346482571823
